In [4]:
# 엘라스틱서치의 데몬 인스턴스 만들기
import os
import json
from elasticsearch import Elasticsearch, helpers
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(['/upstage-ai-advanced-ir7/elasticsearch-8.8.0/bin/elasticsearch'],
                stdout=PIPE, stderr=STDOUT,
                preexec_fn=lambda: os.setuid(1)  # as daemon
                )

# 인스턴스를 로드하는 데 약간의 시간이 걸림
import time
time.sleep(60)

In [7]:
!ps -ef | grep elasticsearch

root      466151  465702  0 14:56 pts/0    00:00:00 sudo -u daemon -s bash -c bash /upstage-ai-advanced-ir7/elasticsearch-8.8.0/bin/elasticsearch
daemon    466152  466151  0 14:56 pts/0    00:00:02 /upstage-ai-advanced-ir7/elasticsearch-8.8.0/jdk/bin/java -Xms4m -Xmx64m -XX:+UseSerialGC -Dcli.name=server -Dcli.script=/upstage-ai-advanced-ir7/elasticsearch-8.8.0/bin/elasticsearch -Dcli.libs=lib/tools/server-cli -Des.path.home=/upstage-ai-advanced-ir7/elasticsearch-8.8.0 -Des.path.conf=/upstage-ai-advanced-ir7/elasticsearch-8.8.0/config -Des.distribution.type=tar -cp /upstage-ai-advanced-ir7/elasticsearch-8.8.0/lib/*:/upstage-ai-advanced-ir7/elasticsearch-8.8.0/lib/cli-launcher/* org.elasticsearch.launcher.CliToolLauncher
daemon    466220  466152  9 14:56 pts/0    00:00:54 /upstage-ai-advanced-ir7/elasticsearch-8.8.0/jdk/bin/java -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -Djava.security.manager=allow -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Df

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
import os
import json
from elasticsearch import Elasticsearch, helpers

es_username = 'elastic'

# 위 명령 실행 결과의 마지막 부분인 PASSWORD elastic 값으로 교체 필요
es_password = 'Yu10AEV5Kj7vhGBRxGeJ'

# Elasticsearch client 생성
es = Elasticsearch(
    ['https://localhost:9200'],
    basic_auth=(es_username, es_password),
    ca_certs="/upstage-ai-advanced-ir7/elasticsearch-8.8.0/config/certs/http_ca.crt")

# Elasticsearch client 정보 확인
print(es.info())

{'name': 'instance-11903', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'yUirY0kNR76abVwg1qeeXg', 'version': {'number': '8.8.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f', 'build_date': '2023-05-23T17:16:07.179039820Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [3]:
settings = {
    "analysis": {
        "analyzer" : {
            "nori": {
                "type" : "custom",
                "tokenizer" : "nori_tokenizer",
                "decompound_mode" : "mixed",
                "filter" : ["nori_posfilter"]
            }
        },
        "filter": {
            "nori_posfilter": {
                "type": "nori_part_of_speech",
                "stoptags" : ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX"]
            }
        }
    }
}

mappings = {
    "properties" : {
        "content": {"type": "text", "analyzer": "nori"},
        "embeddings" : {
            "type": "dense_vector",
            "dims": 768,
            "index": True,
            "similarity": "l2_norm"
        }
    }
}

In [4]:
# 색인 생성
def create_es_index(index, settings, mappings):
    if es.indices.exists(index = index):
        es.indices.delete(index = index)
    
    es.indices.create(index = index, settings = settings, mappings = mappings)


create_es_index("test", settings, mappings)

In [8]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

def get_embedding(sentences):
    return model.encode(sentences)

def get_embeddings_in_batches(docs, batch_size = 100):
    batch_embeddings = []
    for i in tqdm(range(0, len(docs), batch_size)):
        batch = docs[i:i+batch_size]
        contents = [doc["content"] for doc in batch]
        embeddings = get_embedding(contents)
        batch_embeddings.extend(embeddings)

    return batch_embeddings

with open("/upstage-ai-advanced-ir7/data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]

# docs 임베딩
embeddings = get_embeddings_in_batches(docs)

100%|██████████| 43/43 [00:06<00:00,  6.37it/s]


In [9]:
from elasticsearch import helpers

index_docs = []
for doc, embedding in zip(docs, embeddings):
    doc["embeddings"] = embedding.tolist()
    index_docs.append(doc)

def bulk_add(index, docs):
    actions = [
        {
            '_index' : index,
            '_source' : doc
        }
        for doc in docs
    ]

    return helpers.bulk(es, actions)

# 임베딩된 docs를 index에 bulk 단위로 추가
ret = bulk_add("test", index_docs)

In [10]:
test_query = "금성이 다른 행성보다 밝게 보이는 이유는 무엇인가요?"

In [11]:
# 놀라운 사실 이것은 KNN이 아니라 HNSW 알고리즘을 사용하는 것이었습니다.
# knn이라고 주는것은 그저 elasticsearch에서 이름을 그렇게 사용하는 것이 었어요... 소오오오오오름

def dense_retrieve(query_str, size):
    query_embedding = get_embedding([query_str])[0]

    knn = {
        "field" : "embeddings",
        "query_vector" : query_embedding.tolist(),
        "k" : size,
        "num_candidates" : 100
    }

    return es.search(index="test", knn=knn)

def sparse_retrieve(query_str, size):
    query = {
        "match" : {
            "content" : {
                "query" : query_str
            }
        }
    }

    return es.search(index="test", query = query, size = size, sort="_score")

# 역색인 검색 결과(점수 : BM25)
search_result_retrieve = sparse_retrieve(test_query, 3)

for rst in search_result_retrieve['hits']['hits']:
    print(rst)
    print("score: ", rst['_score'], "source: ", rst['_source']['content'])

# KNN 검색 결과
search_result_retrieve = dense_retrieve(test_query, 3)

for rst in search_result_retrieve['hits']['hits']:
    print("socre: ", rst['_score'], "source: ", rst['_source']['content'])

{'_index': 'test', '_id': 'LnkxepIBtyR7svvq8gEd', '_score': 32.46292, '_source': {'docid': '464ace62-ddf2-423d-a5d7-2f17e6785c8e', 'src': 'ko_ai2_arc__ARC_Challenge__train', 'content': '금성이 다른 행성들보다 더 밝게 보이는 이유는 지구 쪽으로 가장 많은 햇빛을 반사하기 때문입니다. 케빈은 맑은 밤에 하늘을 관찰하고 있습니다. 그는 맨눈으로 금성, 화성, 목성, 토성을 볼 수 있습니다. 금성은 햇빛을 많이 반사하기 때문에 다른 행성들보다 더 밝게 보입니다. 이는 금성의 표면이 반사율이 높기 때문입니다. 금성은 태양으로부터 받은 햇빛을 표면에 반사하여 지구에서 관찰하기 쉽게 만듭니다. 따라서 케빈은 맑은 밤에 금성을 더 밝게 볼 수 있습니다.', 'embeddings': [-0.2188829779624939, -0.3961558938026428, -0.4124566316604614, -0.22026996314525604, 0.2667527198791504, -0.6771141886711121, 1.0647681951522827, -0.8683409094810486, 0.312126487493515, -0.2870640158653259, -0.30825507640838623, -0.18074488639831543, 0.03387071192264557, 0.00035873305751010776, -0.22933650016784668, -0.17151087522506714, 0.119020976126194, -0.5617468357086182, 0.42315807938575745, 0.48163336515426636, 0.3518354594707489, 0.4834248125553131, -0.4751303791999817, 0.6908713579177856, 1.074832797050476, -0.6240762472152

# RAG 구현

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("carrotter/ko-gemma-2b-it-sft")
model = AutoModelForCausalLM.from_pretrained("carrotter/ko-gemma-2b-it-sft")

chat = [
    { "role": "user", "content": "피보나치 수열 파이썬 코드로 알려줘" },
]
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=100)
print(tokenizer.decode(outputs[0]))


In [28]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained("rtzr/ko-gemma-2-9b-it")
model = AutoModelForCausalLM.from_pretrained("rtzr/ko-gemma-2-9b-it")

model = model.to(device)

Loading checkpoint shards:  60%|██████    | 6/10 [00:11<00:09,  2.39s/it]

: 

In [ ]:
def generate_search_query(user_input):
    prompt = f"""
    사용자 입력: {user_input}

    당신은 과학 질문을 식별하고 적절한 검색 쿼리를 생성하는 AI 어시스턴트입니다.
    
    1. 주어진 입력이 과학 관련 질문인지 판단하세요.
    2. 과학 질문이라면, Elasticsearch로 구현된 과학 지식 IR 시스템에 전달할 적절한 검색 쿼리를 생성하세요.
    3. 과학 질문이 아니라면, 질문에 적절한 응답하세요.

    응답 형식:
    과학 질문 여부: [예/아니오]
    검색 쿼리: [생성된 쿼리 또는 "해당 없음"]

    예시 응답:
    과학 질문 여부: 예
    검색 쿼리: 광합성 과정 및 필요 요소

    응답:
    """

    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(**inputs, max_length=200, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response.strip()



In [ ]:
# 테스트
questions = [
    "광합성은 어떻게 일어나나요?",
    "오늘 날씨 어때?",
    "원자의 구조에 대해 설명해주세요.",
    "가장 인기있는 영화는 뭐야?"
]

for question in questions:
    result = generate_search_query(question)
    print(f"질문: {question}")
    print(result)
    print("---")

# RAG 구현(Baseline)

In [24]:
persona_qa = """
## Role: 과학 상식 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 겨우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
"""

# 과학 질문인지 아닌지 판별
persona_function_calling = """
## Role: 과학 상식 전문가

## Instruction
- 사용자가 대화를 통해 과학 지식에 관한 주제로 질문하면 search api를 호출할 수 있어야 한다.
- 과학 상식과 관련되지 않은 나머지 대화 메시지에는 적절한 대답을 생성한다.
"""

tools = [
    {
        "type" : "function",
        "function" : {
            "name" : "search",
            "description" : "search relevant documents",
            "parameters" : {
                "properties" : {
                    "standalone_query":{
                        "type" : "string",
                        "description" : "Final query suitable for use in search from the user messages history."
                    }
                },
                "required": ["standalone_query"],
                "type": "object"
            }
        }
    }
]

In [ ]:
def answer_question(messages):
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    msg = [{"role": "system", "content": persona_function_calling}] + messages

    try:
        result = client.chat.completions.create(
            model = llm_model,
            messages = msg,
            tools = tools,
            temperature=0,
            seed=1,
            timeout=10
        )
    except Exception as e:
        traceback.print_exc()
        return response

    # 검색이 필요한 경우
    if result.choices[0].message.tool_calls:
        tool_call = result.choices[0].message.tool_calls[0]
        function_args = json.loads(tool_call.function.arguments) # str -> json으로 만들기
        standalone_query = function_args.get("standalone_query")

        # 검색 하네요
        search_result = sparse_retrieve(standalone_query, 3)

        # 검색 뭐라고 했는지 저장
        response["standalone_query"] = standalone_query

        # 검색 결과 저장
        retrieved_context = []
        for i, rst in enumerate(search_result['hits']['hits']):
            retrieved_context.append(rst["_source"]["content"])
            response["topk"].append(rst["_source"]["content"])
            response["references"].append({"score": rst["_score"], "content": rst["_source"]["content"]})

        # 검색 결과를 message에 붙여서 다시 질문
        content = json.dumps(retrieved_context) # json -> str 로 만들기
        messages.append({"role": "assistant", "content": content})
        msg = [{"role": "system", "content": persona_qa}] + messages

        try:
            qaresult = client.chat.completions.create(
                model = llm_model,
                messgaes = msg,
                temperature = 0,
                seed = 1,
                timeout = 30
            )
        except Exception as e:
            traceback.print_exc()
            return response

        response["answer"] = qaresult.choices[0].message.content

    # 검색이 필요 없는 경우( 과학 질문이 아닌경우)
    else:
        response['answer'] = result.choices[0].message.content

    return response

In [ ]:
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        for line in f:
            j = json.loads(line)
            response = answer_question(j["msg"])

            output = {
                "eval_id": j["eval_id"],
                "standalone_query": response["standalone_query"],
                "topk": response["topk"],
                "answer": response["answer"],
                "references": response["references"]
            }

            of.write(f"{json.dumps(output, ensure_ascil=False)}\n")